In [212]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
import seaborn as sb
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [213]:
data = pd.read_csv('results.csv')
data = data.fillna({"TARGET_ACCOUNT": "UNKNOWN"})
data = data.fillna({"GDDB_ROLE_SPECIAL": "STANDARD"})
data.head()

,USERNAME,EVENT_ID,TARGET_ACCOUNT,TARGET_UNIQUE_ID,TARGET_GENDER,GDDB_ROLE,GDDB_ROLE_SPECIAL,LABEL
0,TROMMRE1,GDDB04,EUNET/BOHNSRO1,BOHNSRO1,Person,ROLE_GDDB_ADMINISTRATOR - none - HELPDESK,STANDARD,incident
1,TROMMRE1,GDDB04,EUNET/KOENIKO1,KOENIKO1,Person,ROLE_GDDB_ADMINISTRATOR - none - HELPDESK,STANDARD,incident
2,COLLAAR1,GDDB04,NANET/CUARTJO1,CUARTJO1,Person,ROLE_GDDB_ADMINISTRATOR - none - HELPDESK,STANDARD,incident
3,DUEBLDO1,GDDB02,EUNET/RELEA201,RELEA201,Person,ROLE_GDDB_MANAGER,STANDARD,exeption
4,DUEBLDO1,GDDB02,EUNET/ADM_RELEA201,RELEA201,Person,ROLE_GDDB_MANAGER,STANDARD,exeption


In [214]:
found = []
for index,dta in enumerate (data['TARGET_ACCOUNT']):
    m = re.search('/(.+?)_', dta)
    if m:
        found.append(m.group(1))
        data['TARGET_ACCOUNT'][index] = m.group(1)
        
ulist = list(set(found))

In [215]:
ta_mapping = dict.fromkeys(ulist)
for u in ulist:
    ta_mapping[u] = ulist.index(u)+1
    
data['TARGET_ACCOUNT'] = data['TARGET_ACCOUNT'].map(ta_mapping)
data = data.fillna({"TARGET_ACCOUNT": 0})

In [216]:
uid_mapping = {'PRASAPU1':1,'CHINKE1':1,'BANNENA3':1}
data['USERNAME'] = data['USERNAME'].map(uid_mapping)
data = data.fillna({"USERNAME": 0})

In [217]:
eventid_mapping = {'GDDB01':0, 'GDDB02':1, 'GDDB03':2, 'GDDB04':3}
data['EVENT_ID'] = data['EVENT_ID'].map(eventid_mapping)

tg_mapping = {'Person':0, 'Generic':1}
data['TARGET_GENDER'] = data['TARGET_GENDER'].map(tg_mapping)

role_mapping = {'ROLE_GDDB_MANAGER':0, 'ROLE_GDDB_ADMINISTRATOR_PLUS':1, 'ROLE_GDDB_ADMINISTRATOR - none - HELPDESK':2, 'ROLE_GDDB_ADMINISTRATOR - none':3}
data['GDDB_ROLE'] = data['GDDB_ROLE'].map(role_mapping)

srole_mapping = {'ROLE_GDDB_AD':0, 'ROLE_GDDB_LEGAL':1, 'STANDARD':2}
data['GDDB_ROLE_SPECIAL'] = data['GDDB_ROLE_SPECIAL'].map(srole_mapping)

label_mapping = {'incident':0, 'exeption':1}
data['LABEL'] = data['LABEL'].map(label_mapping)

In [218]:
data = data.drop('TARGET_UNIQUE_ID',axis=1)

In [219]:
uid = tf.feature_column.numeric_column('USERNAME')
eid = tf.feature_column.numeric_column('EVENT_ID')
ta = tf.feature_column.numeric_column('TARGET_ACCOUNT')
tg = tf.feature_column.numeric_column('TARGET_GENDER')
role = tf.feature_column.numeric_column('GDDB_ROLE')
srole = tf.feature_column.numeric_column('GDDB_ROLE_SPECIAL')
lab = tf.feature_column.numeric_column('LABEL')

In [220]:
data

,USERNAME,EVENT_ID,TARGET_ACCOUNT,TARGET_GENDER,GDDB_ROLE,GDDB_ROLE_SPECIAL,LABEL
0,0.0,3,0.0,0,2.0,2,0
1,0.0,3,0.0,0,2.0,2,0
2,0.0,3,0.0,0,2.0,2,0
3,0.0,1,0.0,0,0.0,2,1
4,0.0,1,15.0,0,0.0,2,1
5,0.0,1,0.0,1,0.0,2,1
6,0.0,3,0.0,0,2.0,2,0
7,0.0,2,0.0,0,0.0,2,1
8,0.0,2,15.0,0,0.0,2,1
9,0.0,2,0.0,1,0.0,2,1


In [221]:
feat_cols = [uid,eid,ta,tg,role,srole]

In [222]:
x_data = data.drop('LABEL',axis=1)

In [223]:
labels = data['LABEL']

In [224]:
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.3, random_state=101)

In [225]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

In [226]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\Gabor\\AppData\\Local\\Temp\\tmprv3oaerd', '_save_summary_steps': 100, '_log_step_count_steps': 100, '_master': '', '_save_checkpoints_steps': None, '_task_id': 0, '_keep_checkpoint_max': 5, '_session_config': None, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_task_type': 'worker', '_is_chief': True, '_tf_random_seed': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FDAFD23F60>}


In [227]:
dnn_model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Gabor\AppData\Local\Temp\tmprv3oaerd\model.ckpt.
INFO:tensorflow:step = 1, loss = 6.28767
INFO:tensorflow:global_step/sec: 419.656
INFO:tensorflow:step = 101, loss = 1.71056 (0.242 sec)
INFO:tensorflow:global_step/sec: 528.654
INFO:tensorflow:step = 201, loss = 0.241763 (0.188 sec)
INFO:tensorflow:global_step/sec: 528.546
INFO:tensorflow:step = 301, loss = 0.170586 (0.191 sec)
INFO:tensorflow:global_step/sec: 507.533
INFO:tensorflow:step = 401, loss = 5.72357 (0.195 sec)
INFO:tensorflow:global_step/sec: 523.834
INFO:tensorflow:step = 501, loss = 0.138875 (0.192 sec)
INFO:tensorflow:global_step/sec: 530.469
INFO:tensorflow:step = 601, loss = 0.0494818 (0.190 sec)
INFO:tensorflow:global_step/sec: 522.904
INFO:tensorflow:step = 701, loss = 0.112957 (0.191 sec)
INFO:tensorflow:global_step/sec: 514.381
INFO:tensorflow:step = 801, loss = 0.0735495 (0.190 sec)
INFO:tensorflow:global_step/sec: 54

In [228]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)

In [229]:
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Starting evaluation at 2017-12-24-00:12:26
INFO:tensorflow:Restoring parameters from C:\Users\Gabor\AppData\Local\Temp\tmprv3oaerd\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-24-00:12:27
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.992288, accuracy_baseline = 0.820051, auc = 0.996865, auc_precision_recall = 0.996865, average_loss = nan, global_step = 1000, label/mean = 0.820051, loss = nan, prediction/mean = nan


{'accuracy': 0.99228793,
 'accuracy_baseline': 0.82005143,
 'auc': 0.99686515,
 'auc_precision_recall': 0.99686521,
 'average_loss': nan,
 'global_step': 1000,
 'label/mean': 0.82005143,
 'loss': nan,
 'prediction/mean': nan}